In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 日誌サンプルデータの読み込み

In [3]:
#encoding='utf-8'で上手くいった
diary_df = pd.read_csv('MFUG_Financial_statement_201803.csv', encoding='utf-8')

In [4]:
diary_df

,text
0,当年度の金融経済環境でありますが、世界経済は、米国の新政権の政策運営や欧州主要国の国政選挙、...
1,米国では、雇用・所得環境の改善に企業の生産活動の持ち直しも加わり、自律的な景気の拡大が続きま...
2,欧州も、英国のＥＵ離脱を巡る不透明感を内在しつつも、良好な雇用・所得環境や低金利等を支えに景...
3,アジアにおいても、先進国の景気拡大に伴う輸出の回復や旺盛なインフラ需要等を支えに、 ＡＳＥＡ...
4,こうした中、我が国経済も、東京都議会議員選挙や衆議院議員総選挙等を巡り 政治が揺れ動く展開と...
5,企業部門では、高水準の企業収益に在庫調整の一巡を受けた生産 の持ち直しも加わり、設備投資の高...
6,家計部門でも、企業の旺盛な採用意欲に支えられた良好 な雇用・所得環境を背景に、個人消費が底堅...
7,加えて、政府による大規模な経済対策の実施も年度 前半にかけての景気を押し上げました。
8,金融情勢に目を転じますと、第３四半期までは、ドル円相場は横這い圏で推移し、世界経済の回復を支...
9,金利については、米国において平成29年６月と12月及び平成30年３ 月に利上げが行われ、ユー...


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [5]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、名詞、形容詞原形と動詞原形を抽出しリストに格納
##### 参考：https://github.com/kujirahand/book-mlearn-gyomu/blob/master/src/ch4/Doc2Vec/create_model.py

In [6]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

# 引数のテキストを分かち書きして配列にする
# node.surface: 文字のみ取得できる 出力例：同  期間  の  SBI  証券
# node.feature: 品詞、原形などの詳細を取得できる:「品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音」の順
# よって、「品詞」を取得したい場合はnode.featureをsplit()でリスト型にした後に抽出したい詳細のインデックス番号[0]を指定すればよい
# また、動詞や形容詞の「原形（の単語）」を取得したい場合はインデックス番号[6]を指定すればよい

def split_words(diary_normalization):
    node = neologd_tagger.parseToNode(diary_normalization) #parseだとエラー「 'str' object has no attribute 'feature'」
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [7]:
wakati_list = []

# 文章の上から順に作成した関数を実行
for diary in diary_df['text']:
    diary_normalization = get_diary_normalization(diary)
    wakati_words = split_words(diary_normalization)
    wakati_list.append(wakati_words)
    
wakati_list

[['当年度',
  '金融経済',
  '環境',
  '世界経済',
  '米国',
  '新政権',
  '政策',
  '運営',
  '欧州',
  '主要',
  '国',
  '国政選挙',
  '中東',
  '北朝鮮情勢',
  '緊張',
  '政策',
  '地政学',
  'リスク',
  'さらす',
  'れる',
  '景気',
  '面',
  '安定',
  'する',
  '回復',
  '拡大',
  '続く'],
 ['米国',
  '雇用',
  '所得',
  '環境',
  '改善',
  '企業',
  '生産',
  '活動',
  '持ち直し',
  '加わる',
  '自律',
  '的',
  '景気',
  '拡大',
  '続く'],
 ['欧州',
  '英国',
  'EU離脱',
  '巡る',
  '不透明感',
  '内在',
  'する',
  '良好',
  '雇用',
  '所得',
  '環境',
  '金利',
  '等',
  '支え',
  '景気',
  '底堅い',
  'さ',
  '維持',
  'する'],
 ['アジア',
  '先進国',
  '景気拡大',
  '伴う',
  '輸出',
  '回復',
  '旺盛',
  'インフラ',
  '需要',
  '等',
  '支え',
  'ASEAN',
  '東南アジア諸国連合',
  'NIEs',
  '新興工業経済地域',
  'はもと',
  '構造',
  '調整',
  '重石',
  '抱える',
  '中国',
  '景気',
  '堅調',
  '推移',
  '示す'],
 ['中',
  '我が国',
  '経済',
  '東京都議会議員選挙',
  '衆議院議員総選挙',
  '等',
  '巡る',
  '政治',
  '揺れ動く',
  '展開',
  'なる',
  '景気',
  '面',
  '景気',
  '拡張',
  '期間',
  'いざなぎ景気',
  '超える',
  '戦後',
  '2位',
  '達する',
  '等',
  '年度',
  '緩やか',
  '拡大',
  '続く'],
 ['企業',
  '部門',
  '高水準',
  '企業',
  '収益

# Doc2Vecの実装

In [8]:
# TaggedDocumentでwords="対象の文章", tags=["タグ"] というように、文章とタグを引数に与える
# 参考URL：https://paiza.hatenablog.com/entry/2017/03/16/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E5%88%9D%E5%BF%83%E8%80%85%E5%90%91%E3%81%91%E3%80%81Word2Vec%E3%81%A8Doc2Vec%E3%81%A7%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0
trainings_wakati_list = [TaggedDocument(words = wakati_list[i], tags = [i]) for i in range(len(diary_df))]
trainings_wakati_list

[TaggedDocument(words=['当年度', '金融経済', '環境', '世界経済', '米国', '新政権', '政策', '運営', '欧州', '主要', '国', '国政選挙', '中東', '北朝鮮情勢', '緊張', '政策', '地政学', 'リスク', 'さらす', 'れる', '景気', '面', '安定', 'する', '回復', '拡大', '続く'], tags=[0]),
 TaggedDocument(words=['米国', '雇用', '所得', '環境', '改善', '企業', '生産', '活動', '持ち直し', '加わる', '自律', '的', '景気', '拡大', '続く'], tags=[1]),
 TaggedDocument(words=['欧州', '英国', 'EU離脱', '巡る', '不透明感', '内在', 'する', '良好', '雇用', '所得', '環境', '金利', '等', '支え', '景気', '底堅い', 'さ', '維持', 'する'], tags=[2]),
 TaggedDocument(words=['アジア', '先進国', '景気拡大', '伴う', '輸出', '回復', '旺盛', 'インフラ', '需要', '等', '支え', 'ASEAN', '東南アジア諸国連合', 'NIEs', '新興工業経済地域', 'はもと', '構造', '調整', '重石', '抱える', '中国', '景気', '堅調', '推移', '示す'], tags=[3]),
 TaggedDocument(words=['中', '我が国', '経済', '東京都議会議員選挙', '衆議院議員総選挙', '等', '巡る', '政治', '揺れ動く', '展開', 'なる', '景気', '面', '景気', '拡張', '期間', 'いざなぎ景気', '超える', '戦後', '2位', '達する', '等', '年度', '緩やか', '拡大', '続く'], tags=[4]),
 TaggedDocument(words=['企業', '部門', '高水準', '企業', '収益', '在庫', '調整', '一巡', '受ける', '生産', '持ち直し',

In [9]:
# Doc2Vecで documents=TaggedDocumet で作成したリスト（今回の場合はtrainings_wakati_list）を指定
# sizeは単語ベクトルの次元数、windowはどの周辺単語まで見るか、min_countは未満の出現頻度の単語は無視する、dm=1はdmpvというアルゴリズムを使う
# https://paiza.hatenablog.com/entry/2017/03/16/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E5%88%9D%E5%BF%83%E8%80%85%E5%90%91%E3%81%91%E3%80%81Word2Vec%E3%81%A8Doc2Vec%E3%81%A7%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0
model = Doc2Vec(documents=trainings_wakati_list, size=300, min_count=1, dm=0)

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [10]:
print(model.docvecs[0])

[ 1.20431825e-03 -8.72437144e-04 -7.19858916e-04  5.82288136e-04
 -1.05578091e-03 -1.28996489e-03 -4.13948183e-05 -1.14231266e-03
  7.80501694e-04  5.11542778e-04  5.17204579e-04 -1.48950692e-03
 -8.39602028e-04 -1.89929095e-04 -1.34000345e-03 -3.43766907e-04
 -1.51368137e-03 -1.33830472e-03 -1.44309422e-03  1.35831756e-03
  7.45999510e-04  1.34706451e-03  4.89648257e-04  8.42080044e-04
 -6.80097437e-04 -1.63197488e-04  7.79874157e-04  2.67828378e-04
  1.10984850e-03 -4.03003534e-04 -5.59858920e-04  1.57546322e-03
 -5.05564676e-04  9.45139967e-04 -1.10511598e-03  3.92184564e-04
  1.06235093e-03 -7.24424375e-04 -6.06231733e-05 -1.21203612e-03
  1.01106113e-03  1.42342155e-03  1.57463917e-04  5.53027203e-04
 -7.67024758e-04 -1.10399059e-03  1.12422497e-03 -6.87585678e-04
 -1.12117536e-03  1.10703474e-03 -7.82781164e-04 -4.76389570e-04
  2.68300646e-04 -6.74379291e-04  1.37450930e-03  4.65907360e-04
 -1.38900313e-03 -6.02057262e-04  1.36416918e-03  9.26377776e-04
  1.22955814e-03  1.45128

In [14]:
for i in range(len(diary_df)):
    print('tag = ' + str(i) +'の文章との類似度\n{}'.format(model.docvecs.most_similar(i)))
    print('-------------------------------------------------------------------------------------')

tag = 0の文章との類似度
[(8, 0.1660725176334381), (9, 0.13208238780498505), (13, 0.1253616213798523), (5, 0.1136433333158493), (7, 0.10438272356987), (16, 0.10097911953926086), (2, 0.10084007680416107), (14, 0.0999135822057724), (11, 0.09619168192148209), (1, 0.08139412105083466)]
-------------------------------------------------------------------------------------
tag = 1の文章との類似度
[(4, 0.13958264887332916), (3, 0.08562449365854263), (0, 0.08139411360025406), (8, 0.057471372187137604), (5, 0.05743465945124626), (13, 0.05475419759750366), (2, 0.041473306715488434), (6, 0.03727986291050911), (9, 0.02499733865261078), (14, 0.017318960279226303)]
-------------------------------------------------------------------------------------
tag = 2の文章との類似度
[(7, 0.18288609385490417), (8, 0.12977835536003113), (16, 0.1157304048538208), (9, 0.1148604080080986), (0, 0.10084007680416107), (11, 0.0805545523762703), (5, 0.07195474207401276), (3, 0.06441972404718399), (6, 0.06097444146871567), (4, 0.0603498332202434

In [13]:
# Doc2Vecの学習モデルを保存
model.save('MFUG_Financial_statement.model')